In [1]:

import os
import sys
import cv2
import numpy as np



image_path=r"C:\Users\gbo10\OneDrive\measurement_paper_images\images used for imageJ\check\stabilized\shai\measurements\1\carapace\right\undistorted_GX010080_238_3392.jpg_gamma.jpg"
image = cv2.imread(image_path)


In [6]:
import cv2
import numpy as np

# Load the image
image = cv2.resize(image, (640, 360))  # Resize the image

# Convert the image to HSV color space
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Define lower and upper bounds for the prawn color range in HSV
lower_bound = np.array([0, 0, 137], dtype="uint8")
upper_bound = np.array([180, 255, 255], dtype="uint8")

# Create a mask using the color bounds
mask = cv2.inRange(hsv, lower_bound, upper_bound)

# Apply the mask to the original image
output = cv2.bitwise_and(image, image, mask=mask)

# Convert the masked output to grayscale
gray_output = cv2.cvtColor(output, cv2.COLOR_BGR2GRAY)

# Apply binary inverse thresholding
_, binary_output = cv2.threshold(gray_output, 127, 255, cv2.THRESH_BINARY_INV)


contours, _ = cv2.findContours(binary_output, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create a copy of the original image to draw on
output_image = image.copy()

# Process each contour
for i, contour in enumerate(contours):
    # Calculate contour area
    area = cv2.contourArea(contour)
    
    # Filter contours based on area (adjust these values as needed)
    if 50 < area < 20000:
        # Get bounding rectangle
        x, y, w, h = cv2.boundingRect(contour)
        
        # Draw rectangle around contour
        cv2.rectangle(output_image, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        # Optionally, crop and save each detected object
        object_roi = image[y:y+h, x:x+w]
        cv2.imwrite(f'object_{i}.png', object_roi)

# Display the result
cv2.imshow('Detected Objects', output_image)
cv2.waitKey(0)

# Save the result
cv2.imwrite('detected_objects.png', output_image)

cv2.destroyAllWindows()


In [4]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('image.png')

# Convert the image to HSV color space
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Define lower and upper bounds for the prawn color range
lower_bound = np.array([0, 0, 137], dtype="uint8")
upper_bound = np.array([180, 255, 255], dtype="uint8")

# Create a mask
mask = cv2.inRange(hsv, lower_bound, upper_bound)

# Apply morphological operations to clean up the mask
# kernel = np.ones((5, 5), np.uint8)
# mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)  # Fill small gaps
# mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)   # Remove small noise

# Save the mask for visualization
cv2.imwrite('cleaned_mask.png', mask)

# Find contours in the cleaned mask
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Filter contours based on area (to ignore very large or very small objects)
min_area = 5000  # Adjust based on prawn size
max_area = 200000  # Adjust based on objects like the checkerboard

# Loop through each contour
for i, contour in enumerate(contours):
    area = cv2.contourArea(contour)

    if area > 5000:

  # Process only contours within the size range
        # Get the bounding box for each prawn
        x, y, w, h = cv2.boundingRect(contour)
        
        # Crop the prawn from the original image
        prawn = image[y:y+h, x:x+w]
        
        # Save each isolated prawn as a separate image
        cv2.imwrite(f'prawn_{i}.png', prawn)

# Optional: Display the final image with bounding boxes
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

cv2.imshow('Final Image with Bounding Boxes', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


# contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# for contour in contours:
    
#     x, y, w, h = cv2.boundingRect(contour)
#     cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

# cv2.imwrite('image.png', image)

In [22]:
# Calculate circularity for each contour and filter by circularity
for contour in contours:
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    if perimeter == 0:
        continue
    circularity = 4 * np.pi * (area / (perimeter * perimeter))
    if circularity > 0.8:  # Threshold for circularity
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)

cv2.imshow('image_with_circular_contours', image)

True

In [2]:
import cv2
import numpy as np

# Load the image
image = cv2.imread(image_path)

# Convert the image to HSV color space
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

cv2.imwrite('hsv_image.jpg', hsv_image)
# Define lower and upper bounds for the carapace color (adjust as needed based on the color)
# Example: if the carapace is brownish, these values will be for brown in HSV.
lower_bound = np.array([90, 0, 0])  # BGR format (Blue, Green, Red)
upper_bound = np.array([129, 97, 121])  # BGR format (Blue, Green, Red)

# Create a mask based on the color range

# 4. Create a mask using the defined color range
mask = cv2.inRange(hsv_image, lower_bound, upper_bound)

# 5. Apply the mask to the original image to isolate the prawns
isolated_prawns = cv2.bitwise_and(image, image, mask=mask)

# 6. Save the segmented image
cv2.imwrite('segmented_prawns.png', isolated_prawns)

# Optional: Display the results
cv2.imshow('Original Image', image)
cv2.imshow('Mask', mask)
cv2.imshow('Isolated Prawns', isolated_prawns)
cv2.waitKey(0)
cv2.destroyAllWindows()



In [3]:
# Define kernel for morphological operations
kernel = np.ones((5, 5), np.uint8)

# Apply morphological opening to remove small noise
mask_cleaned = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

# Apply morphological closing to fill small holes within the objects
mask_cleaned = cv2.morphologyEx(mask_cleaned, cv2.MORPH_CLOSE, kernel)

# Optional: Use Gaussian Blur to smooth the mask edges
mask_blurred = cv2.GaussianBlur(mask_cleaned, (5, 5), 0)

# Apply the cleaned mask to isolate prawns
isolated_prawns = cv2.bitwise_and(image, image, mask=mask_blurred)

# Save the enhanced segmented image
cv2.imwrite('enhanced_segmented_prawns.png', isolated_prawns)

# Optional: Display the enhanced mask and result
cv2.imshow('Cleaned Mask', mask_cleaned)
cv2.imshow('Isolated Prawns - Enhanced', isolated_prawns)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [29]:
import cv2
import numpy as np

def nothing(x):
    pass

# Create a window
cv2.namedWindow('Adjust HSV')

# Create trackbars for lower and upper HSV bounds
cv2.createTrackbar('Lower H', 'Adjust HSV', 0, 180, nothing)
cv2.createTrackbar('Lower S', 'Adjust HSV', 0, 255, nothing)
cv2.createTrackbar('Lower V', 'Adjust HSV', 0, 255, nothing)
cv2.createTrackbar('Upper H', 'Adjust HSV', 180, 180, nothing)
cv2.createTrackbar('Upper S', 'Adjust HSV', 255, 255, nothing)
cv2.createTrackbar('Upper V', 'Adjust HSV', 255, 255, nothing)

# Read the segmented prawn image
prawn_image = cv2.imread(image_path)
resize = cv2.resize(prawn_image, (640, 360))

hsv = cv2.cvtColor(resize, cv2.COLOR_BGR2HSV)

while True:
    # Get current positions of the trackbars
    l_h = cv2.getTrackbarPos('Lower H', 'Adjust HSV')
    l_s = cv2.getTrackbarPos('Lower S', 'Adjust HSV')
    l_v = cv2.getTrackbarPos('Lower V', 'Adjust HSV')
    u_h = cv2.getTrackbarPos('Upper H', 'Adjust HSV')
    u_s = cv2.getTrackbarPos('Upper S', 'Adjust HSV')
    u_v = cv2.getTrackbarPos('Upper V', 'Adjust HSV')

    # Define the lower and upper HSV range
    lower_cara = np.array([l_h, l_s, l_v])
    upper_cara = np.array([u_h, u_s, u_v])

    # Create the mask
    mask = cv2.inRange(hsv, lower_cara, upper_cara)

    # Display the mask
    cv2.imshow('Mask', mask)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

print(f"Final Lower HSV: {lower_cara}")
print(f"Final Upper HSV: {upper_cara}")


Final Lower HSV: [  0   0 137]
Final Upper HSV: [180 255 255]


In [30]:
lower_bound=( 0 ,  0 ,146)
upper_bound=( 180 ,  255 , 255)



mask = cv2.inRange(hsv, lower_bound, upper_bound)

# Morphological operations
kernel = np.ones((3, 3), np.uint8)
mask_cleaned = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=2)
mask_cleaned = cv2.morphologyEx(mask_cleaned, cv2.MORPH_CLOSE, kernel, iterations=2)
mask_cleaned = cv2.dilate(mask_cleaned, kernel, iterations=1)

# Distance Transform
dist_transform = cv2.distanceTransform(mask_cleaned, cv2.DIST_L2, 5)
ret, sure_fg = cv2.threshold(dist_transform, 0.5 * dist_transform.max(), 255, 0)
sure_fg = np.uint8(sure_fg)
sure_bg = cv2.dilate(mask_cleaned, kernel, iterations=3)
unknown = cv2.subtract(sure_bg, sure_fg)

# Marker labeling
ret, markers = cv2.connectedComponents(sure_fg)
markers = markers + 1
markers[unknown == 255] = 0

# Apply Watershed
markers = cv2.watershed(resize, markers)
resize[markers == -1] = [0, 0, 255]  # Optional: mark boundaries in red

# Extract individual prawn masks
prawn_masks = []
for marker in range(2, ret + 2):
    prawn_mask = np.uint8(markers == marker) * 255
    prawn_masks.append(prawn_mask)

print(f"Number of prawns detected: {len(prawn_masks)}")

# Save individual prawns
for i, prawn_mask in enumerate(prawn_masks):
    contours, _ = cv2.findContours(prawn_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours:
        cnt = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(cnt)
        prawn = resize[y:y+h, x:x+w]
        prawn_clean = cv2.bitwise_and(prawn, prawn, mask=prawn_mask[y:y+h, x:x+w])
        cv2.imwrite(f'prawn_{i}.png', prawn_clean)
        print(f"Saved prawn_{i}.png")




Number of prawns detected: 3
Saved prawn_0.png
Saved prawn_1.png


In [25]:
# Define a kernel for morphological operations
kernel = np.ones((5, 5), np.uint8)

# Apply morphological opening to remove small noise
mask_cleaned = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=2)

# Apply morphological closing to fill small holes within the prawns
mask_cleaned = cv2.morphologyEx(mask_cleaned, cv2.MORPH_CLOSE, kernel, iterations=2)

# 


In [26]:
# Convert the cleaned mask to binary
ret, thresh = cv2.threshold(mask_cleaned, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Remove noise
# Sure background area
sure_bg = cv2.dilate(thresh, kernel, iterations=3)

# Finding sure foreground area using distance transform
dist_transform = cv2.distanceTransform(thresh, cv2.DIST_L2, 5)
ret, sure_fg = cv2.threshold(dist_transform, 0.5 * dist_transform.max(), 255, 0)

# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg, sure_fg)

# Marker labeling
ret, markers = cv2.connectedComponents(sure_fg)

# Add one to all labels so that sure background is not 0, but 1
markers = markers + 1

# Mark the unknown regions with zero
markers[unknown == 255] = 0

# Apply watershed
markers = cv2.watershed(resize, markers)

# Boundary marked with -1 after watershed
resize[markers == -1] = [0, 0, 255]  # Optional: Mark boundaries in red

# Display the image with watershed boundaries
cv2.imshow('Watershed Boundaries', resize)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [27]:
# Initialize list to hold individual prawn masks
prawn_masks = []

# Loop through each marker
for marker in range(2, ret + 2):  # markers start from 2
    # Create a mask for each prawn
    prawn_mask = np.uint8(markers == marker) * 255
    prawn_masks.append(prawn_mask)

# Display the number of detected prawns
print(f"Number of prawns detected: {len(prawn_masks)}")


Number of prawns detected: 3


In [28]:
for i, prawn_mask in enumerate(prawn_masks):
    # Find contours for each prawn
    contours, _ = cv2.findContours(prawn_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours:
        # Assuming the largest contour corresponds to the prawn
        cnt = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(cnt)
        
        # Extract the prawn from the original image
        prawn = resize[y:y+h, x:x+w]
        
        # Optionally, apply the mask to get a clean prawn image with background removed
        prawn_clean = cv2.bitwise_and(prawn, prawn, mask=prawn_mask[y:y+h, x:x+w])
        
        # Save the prawn image
        cv2.imwrite(f'prawn_{i}.png', prawn_clean)
        
        print(f"Saved prawn_{i}.png")


Saved prawn_0.png
Saved prawn_1.png


In [ ]:
import cv2
import numpy as np

def isolate_carapace(segmented_image_path, output_path):
    """
    Isolate the carapace from the segmented prawn image.

    Parameters:
    - segmented_image_path: Path to the segmented prawn image.
    - output_path: Path to save the carapace-segmented image.
    """

    # 1. Read the segmented prawn image
    prawn_image = cv2.imread(segmented_image_path)

    if prawn_image is None:
        print(f"Error: Unable to read image at {segmented_image_path}")
        return

    # 2. Convert the image from BGR to HSV color space
    hsv = cv2.cvtColor(prawn_image, cv2.COLOR_BGR2HSV)

    # 3. Define the lower and upper bounds for the carapace color in HSV
    # These values are placeholders and may need adjustment based on your images
    # Example: If the carapace is darker, you might have lower V values
    lower_cara = np.array([0, 0, 0])        # Lower bound of carapace color
    upper_cara = np.array([180, 255, 100])  # Upper bound of carapace color

    # 4. Create a mask using the defined color range
    mask = cv2.inRange(hsv, lower_cara, upper_cara)

    # 5. Refine the mask using morphological operations
    kernel = np.ones((5, 5), np.uint8)

    # Remove small noise
    mask_cleaned = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=2)

    # Fill small holes within the carapace regions
    mask_cleaned = cv2.morphologyEx(mask_cleaned, cv2.MORPH_CLOSE, kernel, iterations=2)

    # Optional: Smooth the edges
    mask_blurred = cv2.GaussianBlur(mask_cleaned, (5, 5), 0)

    # 6. Apply the mask to isolate the carapace
    carapace = cv2.bitwise_and(prawn_image, prawn_image, mask=mask_blurred)

    # 7. Save the segmented carapace image
    cv2.imwrite(output_path, carapace)

    # Optional: Display the results
    cv2.imshow('Original Segmented Prawn', prawn_image)
    cv2.imshow('Carapace Mask', mask_blurred)
    cv2.imshow('Isolated Carapace', carapace)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    # Paths to input and output images
    segmented_image_path = 'segmented_prawns.png'       # Replace with your segmented prawn image path
    carapace_output_path = 'segmented_carapace.png'    # Desired output path for carapace

    isolate_carapace(segmented_image_path, carapace_output_path)
